In [1]:
!pip install --upgrade azureml-automl-core

Requirement already up-to-date: azureml-automl-core in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.21.0)


In [2]:
!pip install --upgrade azureml-sdk[automl]

Requirement already up-to-date: azureml-sdk[automl] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.21.0)


In [4]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.21.0


# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [5]:
import os
import shutil
import joblib

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
ws = Workspace.from_config()

# choose a name for experiment
experiment=Experiment(workspace = ws, name ="hd-experiment")

dataset = Dataset.get_by_name(ws, 'Diabetes-dataset')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-136143
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-136143


## Config Compute Cluster

In [8]:
cpu_cluster = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D12_V2',
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Setting environment for hyperdrive run

In [18]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# install required packages
env = Environment('demo-env')
cd = CondaDependencies.create(pip_packages = ['azureml-dataset-runtime[pandas, fuse]', 
                                               'azureml-default'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

# register environment
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "demo-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
   

In [24]:
from azureml.core import ScriptRunConfig

args = ['--C', 0.5, '--max_iter', 20]
src = ScriptRunConfig(source_directory= '.',
                     script= 'train.py',
                     arguments=args,
                     compute_target=cpu_cluster,
                     environment=env)

In [25]:
run = experiment.submit(config=src)

In [26]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [27]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2, delay_evaluation = 3)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
        "--C": uniform(0.2, 1.0),
        "--max_iter": choice(10, 20, 40)
    })

#TODO: Create your estimator and hyperdrive config
#est = SKLearn(source_directory = '.', 
              #compute_target = cpu_cluster, 
              #entry_script = 'train.py')

hyperdrive_config = HyperDriveConfig(
                                hyperparameter_sampling = ps,
                                policy = policy,
                                primary_metric_name = 'Accuracy',
                                primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs = 20,
                                max_concurrent_runs = 4,
                                run_config = src)

In [28]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_f0c3376a-b341-417d-b55f-f2f96b513628
Web View: https://ml.azure.com/experiments/hd-experiment/runs/HD_f0c3376a-b341-417d-b55f-f2f96b513628?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-136143/workspaces/quick-starts-ws-136143

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-26T20:15:44.532166][API][INFO]Experiment created<END>\n""<START>[2021-01-26T20:15:45.185161][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-26T20:15:45.487531][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-26T20:15:46.3055726Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>
{
  "azureml-logs/20_image_build_log.txt": "https://mlstrg136143.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_f0c3376a-b341-417d-b55f-f2f96b513628_preparation/azureml-logs/20_image_build_log

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Environment preparation run marked as Failed by execution service due to UserError.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Environment preparation run marked as Failed by execution service due to UserError.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)
print('\n Accuracy: ', best_run_metrics['Accuracy'])

In [ ]:
best_run

In [ ]:
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')

In [ ]:
#TODO: Save the best model

In [ ]:
#Register the best model
best_model_registered = best_run.register_model(model_name = 'best_hyperdrive_model', 
                                                model_path='outputs/model.joblib',
                                                description='best hyperdrive model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service